# Example flow for processing and aggregating stats about committee meeting attendees and protocol parts

See the [DataFlows documentation](https://github.com/datahq/dataflows) for more details regarding the Flow object and processing functions.

Feel free to modify and commit changes which demonstrate additional functionality or relevant data.

## Constants

In [169]:
# Limit processing of protocol parts for development
PROCESS_PARTS_LIMIT = 10

# Enable caching of protocol parts data (not efficient, should only be used for local development with sensible PROCESS_PARTS_LIMIT)
PROCESS_PARTS_CACHE = True

# Filter the meetings to be processed, these kwargs are passed along to DataFlows filter_rows processor for meetings resource
MEETINGS_FILTER_ROWS_KWARGS = {'equals': [{'KnessetNum': 20}]}

# Don'e use local data - loads everything from knesset data remote storage
# When set to False - also enables caching, so you won't download from remote storage on 2nd run.
USE_DATA = False

## Loading lexicons

In [170]:
import os

def read_topic_to_set(topic_name):
    lines = open(os.path.join(dir_name, topic_name + ".txt"), 'r').readlines()
    return set([line.strip().replace("\ufeff", "") for line in lines])

dir_name = "../topics/lexicons"        

files = os.listdir(dir_name)

topics = [file.split('.')[0] for file in files]

lexicons = {}
for topic_name in topics:
    lexicons[topic_name] = read_topic_to_set(topic_name)

## Load source data

In [171]:
from dataflows import filter_rows, cache
from datapackage_pipelines_knesset.common_flow import load_knesset_data, load_member_names

# Loads a dict containing mapping between knesset member id and the member name
member_names = load_member_names(use_data=USE_DATA)

# define flow steps for loading the source committee meetings data
# the actual loading is done later in the Flow
load_steps = (
    load_knesset_data('people/committees/meeting-attendees/datapackage.json', USE_DATA),
    filter_rows(**MEETINGS_FILTER_ROWS_KWARGS)
)

if not USE_DATA:
    # when loading from URL - enable caching which will skip loading on 2nd run
    load_steps = (cache(*load_steps, cache_path='.cache/people-committee-meeting-attendees-knesset-20'),)

ERROR   :Internal Python error in the inspect module.
Below is the traceback from this internal error.

INFO    :
Unfortunately, your original traceback can not be constructed.



loading from url: https://storage.googleapis.com/knesset-data-pipelines/data/members/mk_individual/datapackage.json
using cache data from .cache/members-mk-individual-names
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3296, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-171-12b3909e9efe>", line 5, in <module>
    member_names = load_member_names(use_data=USE_DATA)
  File "/pipelines/datapackage_pipelines_knesset/common_flow.py", line 159, in load_member_names
    _load_member_names
  File "/usr/local/lib/python3.6/site-packages/dataflows/base/flow.py", line 15, in process
    return self._chain().process()
  File "/usr/local/lib/python3.6/site-packages/dataflows/base/datastream_processor.py", line 64, in process
    ds = self._process()
  File "/usr/local/lib/python3.6/site-packages/dataflows/base/datastream_processor.py", line 41, in _process
    datastream = self.sou

FileNotFoundError: [Errno 2] No such file or directory

## Inspect the datapackages which will be loaded

Last command's output log should contain urls to datapackage.json files, open them and check the table schema to see the resource metadata and available fields which you can use in the processing functions.

Check the [frictionlessdata docs](https://frictionlessdata.io/docs/) for more details about the datapackage file format.

## Extract topics from lines

In [172]:
from collections import defaultdict
from dataflows import Flow
import pandas as pd
import numpy as np

first = True
running_index = 0

meeting_data_global = None
topics_df = None

stats = defaultdict(int)
member_attended_meetings = defaultdict(int)
rows = []
m_rows = []

def initialize_meeting_data_global(meeting_row):
    global meeting_data_global
    global running_index
    topics_exists = meeting_row['topics'] is not None
    topics = ";".join(meeting_row['topics']) if topics_exists else ""

    meeting_data_global = {
        'KnessetNum': [meeting_row['KnessetNum']],
        'Year': [str(meeting_row['StartDate']).split("-")[0]],
        'CommitteeSessionID': [meeting_row['CommitteeSessionID']],
        'Number': [meeting_row['Number']],
        'Mks': [';'.join(meeting_row['mks'])] if meeting_row['mks'] is not None else [""],
        'Topics': [topics],
        'StartDate': [meeting_row['StartDate']],
        'CommitteeID': [meeting_row['CommitteeID']]
    } 
    
    # Adding topic counts in the 'topics' column 
    if(topics_exists):
        topic_words = topics.split()
        topic_words_size_2 = [" ".join(topic_words[i:i+2]) for i in range(len(topic_words) - 2)]
        topic_words_size_3 = [" ".join(topic_words[i:i+3]) for i in range(len(topic_words) - 3)]
      
    for topic_name, lexicon in lexicons.items():
        if not topics_exists:
            meeting_data_global[topic_name + "_score"] = [0]
        else:
            count = lexicon_count(lexicon, topic_words) + lexicon_count(lexicon, topic_words_size_2) + lexicon_count(lexicon, topic_words_size_3)
            meeting_data_global[topic_name + "_score"] = [count*3]
    
def word_permutations(word):
    clean_word = word.strip()
    permutations = [clean_word]
    if len(word) > 1 and word.startswith('ה') or word.startswith('ב') or word.startswith('ל'):
        permutations.append(word[1:])
    return permutations


def in_lexicon(word, lexicon):
    for p in word_permutations(word):
        if p in lexicon:
            return True
    return False

            
def lexicon_count(lexicon, words):
    count = 0
    for word in words:
        if in_lexicon(word, lexicon):
            count += 1
    return count   


def process_meeting_protocol_part(row):
    global meeting_data_global
    global running_index
    global rows
    rows.append(row)
    stats['processed parts'] += 1
    words = row['header'].split() if row['header'] is not None else []
    words += row['body'].split() if row['body'] is not None else []
    words_size_2 = [" ".join(words[i:i+2]) for i in range(len(words) - 2)]
    words_size_3 = [" ".join(words[i:i+3]) for i in range(len(words) - 3)]
                    
    for topic_name, lexicon in lexicons.items():
        meeting_data_global[topic_name + "_score"][0] += lexicon_count(lexicon, words)
        meeting_data_global[topic_name + "_score"][0] += lexicon_count(lexicon, words_size_2)  
        meeting_data_global[topic_name + "_score"][0] += lexicon_count(lexicon, words_size_3)  
            

def process_meeting(row):
    global topics_df
    global running_index
    
    m_rows.append(row)
    stats['total meetings'] += 1
    if row['attended_mk_individual_ids']:
        for mk_id in row['attended_mk_individual_ids']:
            member_attended_meetings[mk_id] += 1
    parts_filename = row['parts_parsed_filename']
    if parts_filename:
        if PROCESS_PARTS_LIMIT and stats['processed parts'] < PROCESS_PARTS_LIMIT:
            initialize_meeting_data_global(row)
            steps = (load_knesset_data('committees/meeting_protocols_parts/' + parts_filename, USE_DATA),)
            if not USE_DATA and PROCESS_PARTS_CACHE:
                steps = (cache(*steps, cache_path='.cache/committee-meeting-protocol-parts/' + parts_filename),)
            steps += (process_meeting_protocol_part,)
            Flow(*steps).process()
            running_index += 1

def add_meeting_data_to_table(row):
    global topics_df
    global meeting_data_global
    max_score = 0
    max_lexicon = "None"
    for lexicon_name in lexicons:
        score = meeting_data_global[lexicon_name+"_score"]
        if score > max_score:
            max_score = score
            max_lexicon = lexicon_name
            
    meeting_data_global['BestTopic'] = [max_lexicon]
    if topics_df is None:
        topics_df = pd.DataFrame(meeting_data_global)
    else:
        topics_df = topics_df.append(pd.DataFrame(meeting_data_global), ignore_index=True)


process_steps = (
    process_meeting,
    add_meeting_data_to_table,
)

In [173]:
from dataflows import Flow, dump_to_path

Flow(*load_steps, *process_steps, dump_to_path('data/committee-meeting-attendees-parts')).process()

ERROR   :Internal Python error in the inspect module.
Below is the traceback from this internal error.

INFO    :
Unfortunately, your original traceback can not be constructed.



using cache data from .cache/people-committee-meeting-attendees-knesset-20
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3296, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-173-ae4001ba7d74>", line 3, in <module>
    Flow(*load_steps, *process_steps, dump_to_path('data/committee-meeting-attendees-parts')).process()
  File "/usr/local/lib/python3.6/site-packages/dataflows/base/flow.py", line 15, in process
    return self._chain().process()
  File "/usr/local/lib/python3.6/site-packages/dataflows/base/datastream_processor.py", line 64, in process
    ds = self._process()
  File "/usr/local/lib/python3.6/site-packages/dataflows/base/datastream_processor.py", line 41, in _process
    datastream = self.source._process()
  File "/usr/local/lib/python3.6/site-packages/dataflows/base/datastream_processor.py", line 41, in _process
    datastream = self.source._process()
  File

FileNotFoundError: [Errno 2] No such file or directory

In [148]:
df = topics_df

In [149]:
df['Year'] = df['StartDate'].apply(lambda l: str(l).split("-")[0])
df['BestTopic'] = df[['Diplomacy_score', 'Ecologics_score', 'Economics_score','Education_score', 'Health_score', 'Security_score']].idxmax(axis=1)
df

,KnessetNum,CommitteeSessionID,Number,Mks,Topics,StartDate,CommitteeID,Diplomacy_score,Ecologics_score,Economics_score,Education_score,Health_score,Security_score,Year,BestTopic
0,20,562716,1,"זאב אלקין – יו""ר הוועדה המסדרת;ניסן סלומינסקי ...",בחירת יושב-ראש לוועדת הכספים הזמנית,2015-04-01 14:00:00,922,0,0,11,0,0,0,2015,Economics_score
1,20,562716,1,"זאב אלקין – יו""ר הוועדה המסדרת;ניסן סלומינסקי ...",בחירת יושב-ראש לוועדת הכספים הזמנית,2015-04-01 14:00:00,922,0,0,11,0,0,0,2015,Economics_score
2,20,562716,1,"זאב אלקין – יו""ר הוועדה המסדרת;ניסן סלומינסקי ...",בחירת יושב-ראש לוועדת הכספים הזמנית,2015-04-01 14:00:00,922,0,0,11,0,0,0,2015,Economics_score
3,20,562716,1,"זאב אלקין – יו""ר הוועדה המסדרת;ניסן סלומינסקי ...",בחירת יושב-ראש לוועדת הכספים הזמנית,2015-04-01 14:00:00,922,0,0,11,0,0,0,2015,Economics_score
4,20,562716,1,"זאב אלקין – יו""ר הוועדה המסדרת;ניסן סלומינסקי ...",בחירת יושב-ראש לוועדת הכספים הזמנית,2015-04-01 14:00:00,922,0,0,11,0,0,0,2015,Economics_score
5,20,562716,1,"זאב אלקין – יו""ר הוועדה המסדרת;ניסן סלומינסקי ...",בחירת יושב-ראש לוועדת הכספים הזמנית,2015-04-01 14:00:00,922,0,0,11,0,0,0,2015,Economics_score
6,20,562716,1,"זאב אלקין – יו""ר הוועדה המסדרת;ניסן סלומינסקי ...",בחירת יושב-ראש לוועדת הכספים הזמנית,2015-04-01 14:00:00,922,0,0,11,0,0,0,2015,Economics_score
7,20,562716,1,"זאב אלקין – יו""ר הוועדה המסדרת;ניסן סלומינסקי ...",בחירת יושב-ראש לוועדת הכספים הזמנית,2015-04-01 14:00:00,922,0,0,11,0,0,0,2015,Economics_score
8,20,562716,1,"זאב אלקין – יו""ר הוועדה המסדרת;ניסן סלומינסקי ...",בחירת יושב-ראש לוועדת הכספים הזמנית,2015-04-01 14:00:00,922,0,0,11,0,0,0,2015,Economics_score
9,20,562716,1,"זאב אלקין – יו""ר הוועדה המסדרת;ניסן סלומינסקי ...",בחירת יושב-ראש לוועדת הכספים הזמנית,2015-04-01 14:00:00,922,0,0,11,0,0,0,2015,Economics_score


In [158]:
g = df[['Year', 'BestTopic']].groupby(['Year', 'BestTopic']).count()

In [159]:
g.describe()

ERROR   :Internal Python error in the inspect module.
Below is the traceback from this internal error.

INFO    :
Unfortunately, your original traceback can not be constructed.



Traceback (most recent call last):
  File "/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3296, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-159-f6fff0ef2c4b>", line 1, in <module>
    g.describe()
  File "/usr/local/lib/python3.6/site-packages/pandas/core/generic.py", line 9721, in describe
    raise ValueError("Cannot describe a DataFrame without columns")
ValueError: Cannot describe a DataFrame without columns

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2033, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'ValueError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/site-packages/IPython/core/ultratb.py", lin

ValueError: Cannot describe a DataFrame without columns

In [ ]:
with open("topics_output.csv", 'w') as f:
    f.write(topics_df.to_csv())

## Aggregate and print stats

In [113]:
from collections import deque
import yaml

top_attended_member_names = [member_names[mk_id] for mk_id, num_attended in
                             deque(sorted(member_attended_meetings.items(), key=lambda kv: kv[1]), maxlen=5)]
print('\n')
print('-- top attended members --')
print(top_attended_member_names)
print('\n')
print('-- stats --')
print(yaml.dump(dict(stats), default_flow_style=False, allow_unicode=True))



-- top attended members --
['איתן ברושי', 'לוי מיקי', 'דוב חנין', 'רפאל משה גפני', 'אורי מקלב']


-- stats --
processed parts: 42
total meetings: 10256



## Get output data

Output data is available in the left sidebar under data directory, you can check the datapackage.json and created csv file to explore the data and schema.